In [38]:
import csv, pandas as pd
# Make a list of all the expected column names for the documents
column_names = ['Url', 'Date', 'Content', 'Id', 'User', 'Reply Count',
       'Retweet Count', 'Like Count', 'Quote Count', 'Conversation Id',
       'Language', 'Source', 'Coordinates']
# Convert all the names to lowercase to make it easier in the future.
for index, name in enumerate(column_names):
       column_names[index] = name.lower()

In [39]:
# List to hold each groups documents
all_docs = []
# Iterate 5 times in order load in all the groups csv files
for index in range(1,6):
    # Build the path for the current document
    current_file = f"../all_dirty_docs/tweets_{index}.csv"
    # Load and save the document into a variable
    this_document = pd.read_csv(filepath_or_buffer=current_file)
    # Create a dictionary that has
    # The column name as a key, and the lowered column name as a value
    column_dict = {}
    for name in this_document.columns.values:
        column_dict[name] = name.lower()
    # Rename all the columns to their lowered version
    this_document.rename(columns=column_dict,inplace=True)
    # Add the document to the tweets_list list.
    all_docs.append(this_document)
    # First, check to see if the number of columns matches.
    print(len(this_document.columns.values), " vs. ",len(column_names))
    

14  vs.  13
14  vs.  13
12  vs.  13
14  vs.  13
14  vs.  13


In [40]:
#%%script echo skipping
for x in range(0,5):
    print(all_docs[x].columns)

Index(['unnamed: 0', 'url', 'date', 'content', 'user', 'replies', 'retweets',
       'likes', 'quotes', 'id', 'language', 'source', 'coordinates', 'dashes'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'replycount', 'retweetcount',
       'likecount', 'quotecount', 'conversationid', 'lang', 'source',
       'coordinates', 'unnamed: 13'],
      dtype='object')
Index(['date', 'user', 'tweet', 'likes', 'id', 'reply count', 'retweet count',
       'quote count', 'conversation id', 'language', 'source', 'coordinates'],
      dtype='object')
Index(['unnamed: 0', 'url', 'date', 'content', 'id', 'user', 'reply count',
       'retweet count', 'like count', 'quote count', 'conversation id',
       'language', 'source', 'coordinates'],
      dtype='object')
Index(['unnamed: 0', 'url', 'date', 'content', 'id', 'user', 'reply count',
       'retweet count', 'like count', 'quote count', 'conversation id', 'lang',
       'source', 'coordinates'],
      dtype='object')


In [41]:
# Figure out what the maximum "ID" is for a frame.
# Before we can do this we need to determine what column is the 
# bugged "ID" column.
# In the case of tweets_1, it is the column "unnamed: 0"
# Therefore, any document with a column named "unnamed: 0"
# will need to undergo this process

# REFACTOR THIS SO I CAN CALL IT AS A FUNCTION AND IT WILL TEST
# ALL DOCUMENTS WITHIN 
header_docs = [all_docs[0],all_docs[3],all_docs[4]]
my_max = 0
for doc in header_docs:
    print(f"Num of rows before: {doc.shape[0]}")
    # Removes all instances of header columns by removing any row where "Unnamed: 0" == NaN
    doc.dropna(subset=["unnamed: 0"], inplace=True)
    # Now remove the column all together and re-index the data frame.
    doc.drop(labels="unnamed: 0",axis=1,inplace=True)
    doc.reset_index(inplace=True,drop=True)
    # Finally, drop the created "index" column.
    print(f"Num of rows after: {doc.shape[0]}")
all_docs[1].drop(labels="unnamed: 13",axis=1,inplace=True)
    

# Following this the documents 1 and 5 should be fixed and correctly indexed
# and stripped of the problem columns
#
# We still need to fix document 4 however.

# Same thing - Reminder
# print(type(tweets_list[0].Url))
# print(type(tweets_list[0]["Url"]))

Num of rows before: 1031
Num of rows after: 1031
Num of rows before: 1208
Num of rows after: 1200
Num of rows before: 1200
Num of rows after: 1200


In [42]:
for x in range(0,5):
    print(all_docs[x].columns)

Index(['url', 'date', 'content', 'user', 'replies', 'retweets', 'likes',
       'quotes', 'id', 'language', 'source', 'coordinates', 'dashes'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'replycount', 'retweetcount',
       'likecount', 'quotecount', 'conversationid', 'lang', 'source',
       'coordinates'],
      dtype='object')
Index(['date', 'user', 'tweet', 'likes', 'id', 'reply count', 'retweet count',
       'quote count', 'conversation id', 'language', 'source', 'coordinates'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'lang', 'source',
       'coordinates'],
      dtype='object')


In [57]:
#%%script echo skipping
# Now that those problem documents are fixed we need to make all the tables
# Have the same column names and order of names.
# Then find problem columns by checking the column names
# against our expected column names.
for index, doc in enumerate(all_docs):
    rename_list = []
    # For dataframe.rename we need a dict with key = current name
    # value = desired name
    for name in column_names:
        if name not in doc.columns.values:
            print(f"Document {index+1} missing column \"{name}\"")

Document 1 missing column "reply count"
Document 1 missing column "retweet count"
Document 1 missing column "like count"
Document 1 missing column "quote count"
Document 1 missing column "conversation id"
Document 2 missing column "reply count"
Document 2 missing column "retweet count"
Document 2 missing column "like count"
Document 2 missing column "quote count"
Document 2 missing column "conversation id"
Document 2 missing column "language"
Document 3 missing column "url"
Document 3 missing column "content"
Document 3 missing column "like count"
Document 5 missing column "language"


In [55]:
#This code here is to get rid of all of the twitter handles, emojis and links
import re #regex for the link and handle detection
import demoji 

#This part is a proof of conecpt that we are able to remove links, emojis and twitter handles
for row in corpus.index[0:5]: #iterate through each row of the corpus 
    content = doc['content'][row]
    print(content)
    #sub the twitter handles 
    content = re.sub("@[a-z]*","xxhandle", content)
    print(content)
    #sub out the website links
    content = re.sub('http\S+', 'xxhyperlink', content)
    print(content)
    #sub out emojis
    content = demoji.replace(content, "xxemoji")
    print(content)

#now to actually do it and update the data in the df
    
for row in corpus.index:
    content = doc['content'][row]
    content = re.sub("@[a-z]*","xxhandle", content)
    content = re.sub('http\S+', 'xxhyperlink', content)
    content = demoji.replace(content, "xxemoji")
    doc.at['content', row] = content

display(doc)
   

xxhandle Amazing. Keep going Mick xxemoji
xxhandle Amazing. Keep going Mick xxemoji
xxhandle Amazing. Keep going Mick xxemoji
xxhandle Amazing. Keep going Mick xxemoji
xxhandleMcIlroyRory Class mate xxemojixxemojixxemoji
xxhandleMcIlroyRory Class mate xxemojixxemojixxemoji
xxhandleMcIlroyRory Class mate xxemojixxemojixxemoji
xxhandleMcIlroyRory Class mate xxemojixxemojixxemoji
xxhandleCH14_ Great to see you mate. Special memories xxemoji
xxhandleCH14_ Great to see you mate. Special memories xxemoji
xxhandleCH14_ Great to see you mate. Special memories xxemoji
xxhandleCH14_ Great to see you mate. Special memories xxemoji
Great start to pre-season. Lads have adapted themselves well and lots of positives to build on. Looking forward to the upcoming games. xxemoji️xxemoji xxhandle xxhyperlink
Great start to pre-season. Lads have adapted themselves well and lots of positives to build on. Looking forward to the upcoming games. xxemoji️xxemoji xxhandle xxhyperlink
Great start to pre-season. L

,url,date,content,id,user,reply count,retweet count,like count,quote count,conversation id,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,https://twitter.com/WayneRooney/status/1620676...,2023-02-01 06:53:01+00:00,xxhandle Amazing. Keep going Mick xxemoji,1.620677e+18,https://twitter.com/WayneRooney,7.0,7.0,643.0,0.0,1.620487e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://twitter.com/WayneRooney/status/1620138...,2023-01-30 19:13:27+00:00,xxhandleMcIlroyRory Class mate xxemojixxemojix...,1.620138e+18,https://twitter.com/WayneRooney,5.0,5.0,416.0,0.0,1.620092e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://twitter.com/WayneRooney/status/1618343...,2023-01-25 20:22:55+00:00,xxhandleCH14_ Great to see you mate. Special m...,1.618344e+18,https://twitter.com/WayneRooney,9.0,25.0,1315.0,1.0,1.618023e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://twitter.com/WayneRooney/status/1618342...,2023-01-25 20:17:20+00:00,Great start to pre-season. Lads have adapted t...,1.618342e+18,https://twitter.com/WayneRooney,20.0,84.0,2792.0,1.0,1.618342e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://twitter.com/WayneRooney/status/1612944...,2023-01-10 22:48:49+00:00,xxhandleMarcusRashford Class xxhandleMarcusRas...,1.612945e+18,https://twitter.com/WayneRooney,196.0,1809.0,36533.0,86.0,1.612941e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,https://twitter.com/tedcruz/status/16179860920...,2023-01-24 20:41:59+00:00,Does the White House think the American people...,1.617986e+18,https://twitter.com/tedcruz,1750.0,1132.0,6411.0,107.0,1.617986e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,https://twitter.com/tedcruz/status/16179606979...,2023-01-24 19:01:05+00:00,The double standard is obvious. #Verdict \n\nx...,1.617961e+18,https://twitter.com/tedcruz,89.0,51.0,345.0,2.0,1.617961e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,https://twitter.com/tedcruz/status/16179606817...,2023-01-24 19:01:01+00:00,xxemoji,1.617961e+18,https://twitter.com/tedcruz,75.0,124.0,586.0,4.0,1.617961e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1199,https://twitter.com/tedcruz/status/16179269545...,2023-01-24 16:47:00+00:00,The American people are ticked off— they don’t...,1.617927e+18,https://twitter.com/tedcruz,2258.0,882.0,5604.0,151.0,1.617927e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# With the previous cell we make sure all column names are present.
# Now we need to reorder them
corpus = pd.concat(all_docs)
corpus.drop(labels="dashes",axis=1,inplace=True)
display(corpus)

,url,date,content,user,replies,retweets,likes,quotes,id,language,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,https://twitter.com/KAKA/status/13258032030243...,2020-11-09 14:11:30+00:00,@igor_fred @JoeBiden Eu moro no Brasil. rsrsrs...,https://twitter.com/KAKA,4.0,0.0,62.0,0.0,1325801834221621248.0,pt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://twitter.com/KAKA/status/13258026540486...,2020-11-09 14:11:30+00:00,@igor_fred @JoeBiden Eu moro no Brasil. rsrsrs...,https://twitter.com/KAKA,4.0,0.0,62.0,0.0,1325801834221621248.0,pt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://twitter.com/KAKA/status/13258018342216...,2020-11-09 14:06:04+00:00,"Não, não se preocupe que esse conteúdo não é p...",https://twitter.com/KAKA,44.0,14.0,704.0,3.0,1325801834221621248.0,pt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://twitter.com/KAKA/status/13226056372814...,2020-10-31 18:25:31+00:00,Which one is your favorite?,https://twitter.com/KAKA,93.0,61.0,1992.0,46.0,1325801834221621248.0,pt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://twitter.com/KAKA/status/13200313853195...,2020-10-24 15:56:21+00:00,Vamosssss 🚀🚀🚀 #HalaMadrid #ElClasico,https://twitter.com/KAKA,667.0,1497.0,24152.0,356.0,1322605637281456128.0,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,https://twitter.com/tedcruz/status/16179860920...,2023-01-24 20:41:59+00:00,Does the White House think the American people...,https://twitter.com/tedcruz,NaN,NaN,NaN,NaN,1617986092036415488.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,https://twitter.com/tedcruz/status/16179606979...,2023-01-24 19:01:05+00:00,The double standard is obvious. #Verdict \n\nx...,https://twitter.com/tedcruz,NaN,NaN,NaN,NaN,1617960697908064256.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,https://twitter.com/tedcruz/status/16179606817...,2023-01-24 19:01:01+00:00,xxemoji,https://twitter.com/tedcruz,NaN,NaN,NaN,NaN,1617960681793523712.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1199,https://twitter.com/tedcruz/status/16179269545...,2023-01-24 16:47:00+00:00,The American people are ticked off— they don’t...,https://twitter.com/tedcruz,NaN,NaN,NaN,NaN,1617926954576904192.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
